In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re

In [ ]:
df_before = pd.read_csv('~/Documents/GitHub/2022-Pergi-Kuliner-Restaurant-Data/pergikuliner_restaurants_dataset.csv')
df_before.head()

In [ ]:
#cleared
#col 1, 2, 3, 9, 10 and 12 do not have nans.
#col 4: fixed branch data does not exist and the columns got filled with other information. 
#col 5: cleared. look precise enough, removed the quotation marks.
#col 11: fixed 57 phone number nans. some of these are just not scraped.
#col 13: fixed bot does not scrape rating when it goes below 4.
#col4: fixed put 'kapasitas kursi' in the right cols.

#awaiting script rerun
#col 6: promotional texts does not get scraped correctly. added str() to get_text() method but the script needs to be rerun.
#col 8: schedule does not get scraped correctly. added str() to get_text() method but the script needs to be rerun.

In [ ]:
chromeOptions = Options()
chromeOptions.headless = True
browserDriver = webdriver.Chrome(ChromeDriverManager().install(), options=chromeOptions)

bucket = []
for u, n in zip(df_before['3'], range(len(df_before))):
    print(f'retrieving from {u}, {n}th website')
    browserDriver.get(u)
    soup = BeautifulSoup(browserDriver.page_source)
    
    results = soup.find(attrs={'itemprop': 'openingHours'}).get_text() if not None else None
    results = soup.find(attrs={'class': 'promotional-text'}).get_text() if not None else None
    results = soup.find(attrs ={'href' : re.compile('tel:')}).get_text()
    results = ','.join([i.getText().strip() for i in soup.find_all(class_='rate-box-bottom')])

    bucket.append(pd.Series(subbucket))

browserDriver.close()

df = pd.concat(bucket)

In [ ]:
chromeOptions = Options()
chromeOptions.headless = True
browserDriver = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver", options=chromeOptions)

results = []
for u, n in zip(df_before['3'], range(len(df_before))):
    print(f'retrieving from {u}, {n}th website, {round(n/1500*100,1)}%                              ', end="\r")
    browserDriver.get(u)
    soup = BeautifulSoup(browserDriver.page_source)

    itemdict = {'itemprop': 'openingHours', 'class': 'promotional-text', 'href' : re.compile('tel:')}
    resultbucket = []
    for k, v in itemdict.items():
        tag = soup.find(attrs={k: v})
        if tag is not None:
            result = tag.get_text() 
        else:
            result = None
        resultbucket.append(result)
        
    ratings = ','.join([i.getText().strip() for i in soup.find_all(class_='rate-box-bottom')])
    resultbucket.append(ratings)

results.append(resultbucket)

browserDriver.close()

In [ ]:
# fixed put 'kapasitas kursi' in the right cols.
pd.set_option('max_colwidth', 120)
df['14'] = df_before.loc[df_before['4'].str.contains('Kapasitas'), '4']

In [ ]:
#fixed branch data does not exist and the columns got filled with other information
df['4'] = df['4'].str.contains('Cabang: ya')

In [ ]:
# fixed 57 phone number nans. some of these are just not scraped.
def retrieve_phone(ulink):
    browser = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver")
    browser.get(ulink)
    soup = BeautifulSoup(browser.page_source)
    phone = str(soup.find(href=re.compile('tel:')).get_text())
    browser.close()
    return phone

missing_phone_url = df.loc[df['11'].isna(), '3']

for nums in missing_phone_url.index:
    missing_phone_url[nums] = retrieve_phone(missing_phone_url[nums])

df.loc[df['11'].isna(), '11'] = missing_phone_url

In [ ]:
# fixed bot does not scrape rating when it goes below 4.
def retrieve_ratings(ulink):
    browser = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver")
    browser.get(ulink)
    soup = BeautifulSoup(browser.page_source)
    phone = ','.join([i.getText().strip() for i in soup.find_all(class_='rate-box-bottom')])
    browser.close()
    return phone

missing_ratings = df.loc[df.iloc[:,13].str.len() < 19, '13']

for idx in missing_ratings.index:
    missing_ratings[idx] = retrieve_ratings(df.loc[idx, '3'])

df.loc[df.iloc[:,13].str.len() < 19, '13'] = missing_ratings

In [ ]:
#checks
df['13'].str.len().value_counts()

In [ ]:
#checks
missing_phone_url

In [ ]:
#checks
df['5'].str.replace('"', '').str.split(',', expand=True).stack().value_counts()

In [ ]:
df['4'].value_counts()

In [ ]:
# fixed
df['5'] = df['5'].str.replace('"', '')

In [ ]:
['Name', 'culinary_type', 'address', 'url', '']